In [1]:
import ast

In [2]:
import pandas

In [ ]:
pandas.read_csv("data.csv")

In [3]:
tree = ast.parse('pandas.read_csv("data.csv")')

In [4]:
tree

In [9]:
!pip install astor 

In [21]:
import ast
from rdflib import Graph, BNode, Literal, RDF, URIRef

def ast_to_rdf(code):
    # Parse the code into an AST
    tree = ast.parse(code)

    # Create a new RDF graph
    g = Graph()

    # Define a recursive function to traverse the AST and add nodes to the graph
    def add_node(node, parent=None):
        # Create a new blank node for this AST node
        bnode = BNode()

        # Add a triple to the graph for this node's type
        g.add((bnode, RDF.type, Literal(type(node).__name__)))

        # If this node has a parent, add a triple for the parent-child relationship
        if parent is not None:
            g.add((parent, URIRef('hasChild'), bnode))

        # If this node has source code, add a triple for the source code
        if hasattr(node, 's'):
            g.add((bnode, URIRef('hasSourceCode'), Literal(node.s)))

        # Recursively add this node's children to the graph
        for child in ast.iter_child_nodes(node):
            add_node(child, parent=bnode)

    # Add the root of the AST to the graph
    add_node(tree)

    # Return the RDF graph
    return g


# Example usage:
code = 'pandas.read_csv(p)'
g = ast_to_rdf(code)

# Print out the triples in the graph
for s, p, o in g:
    print(s, p, o)

In [19]:
!pip install pyvis

In [22]:
from pyvis.network import Network
import networkx as nx

# Create a NetworkX graph from the RDF graph
G = nx.Graph()

# Add nodes and edges to the NetworkX graph
for s, p, o in g:
    G.add_edge(str(s), str(o), title=str(p))

# Create a PyVis network from the NetworkX graph
net = Network(notebook=True)
net.from_nx(G)

# Show the network
net.show("graph.html")